<a href="https://colab.research.google.com/github/sarathkumar1304/Spotify_Music_Recommendation_system/blob/main/Music_Recommdenation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.1/241.1 kB 5.6 MB/s eta 0:00:00


In [1]:
client_id ='3eb82a6fac8c4db58e1bf09fdeb51446'
client_secrte_id='af9af0da07524fab8f0caabc62f64980'

In [2]:
import requests
import base64

In [3]:
# Replace with your own Client ID and Client Secret
CLIENT_ID = client_id
CLIENT_SECRET = client_secrte_id

In [4]:
# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

In [5]:
# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [8]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [9]:
def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [12]:
playlist_id = '37i9dQZF1DX76Wlfdnj7AP'

In [13]:
# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

                               Track Name  \
0                         I'm Good (Blue)   
1                      Boy's a Liar Pt. 2   
2   Quevedo: Bzrp Music Sessions, Vol. 52   
3                         Me Porto Bonito   
4                             El Merengue   
..                                    ...   
95                               PUNTO 40   
96                   Marianela (Que Pasa)   
97                   Lionheart (Fearless)   
98      family ties (with Kendrick Lamar)   
99              Levitating (feat. DaBaby)   

                                     Artists  \
0                   David Guetta, Bebe Rexha   
1                  PinkPantheress, Ice Spice   
2                          Bizarrap, Quevedo   
3                Bad Bunny, Chencho Corleone   
4                  Marshmello, Manuel Turizo   
..                                       ...   
95                Rauw Alejandro, Baby Rasta   
96  HUGEL, Merk & Kremont, Lirico En La Casa   
97                   Joel C

In [14]:
print(music_df.isnull().sum())

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
data = music_df

In [18]:
data.head()

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,I'm Good (Blue),"David Guetta, Bebe Rexha",I'm Good (Blue),7M842DMhYVALrXsw3ty7B3,4uUG5RXrOk84mYEfFvj3cK,94,2022-08-26,175238,True,https://open.spotify.com/track/4uUG5RXrOk84mYE...,...,0.965,7,-3.673,0,0.0343,0.00383,0.000007,0.3710,0.304,128.040
1,Boy's a Liar Pt. 2,"PinkPantheress, Ice Spice",Boy's a liar Pt. 2,6cVfHBcp3AdpYY0bBglkLN,6AQbmUe0Qwf5PZnt4HmTXv,93,2023-02-03,131013,False,https://open.spotify.com/track/6AQbmUe0Qwf5PZn...,...,0.809,5,-8.254,1,0.0500,0.25200,0.000128,0.2480,0.857,132.962
2,"Quevedo: Bzrp Music Sessions, Vol. 52","Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",4PNqWiJAfjj32hVvlchV5u,2tTmW7RDtMQtBk7m2rYeSw,92,2022-07-06,198938,False,https://open.spotify.com/track/2tTmW7RDtMQtBk7...,...,0.782,2,-5.548,1,0.0440,0.01250,0.033000,0.2300,0.550,128.033
3,Me Porto Bonito,"Bad Bunny, Chencho Corleone",Un Verano Sin Ti,3RQQmkQEvNCY4prGKE6oc5,6Sq7ltF9Qa7SNFBsV5Cogx,92,2022-05-06,178567,True,https://open.spotify.com/track/6Sq7ltF9Qa7SNFB...,...,0.712,1,-5.105,0,0.0817,0.09010,0.000027,0.0933,0.425,92.005
4,El Merengue,"Marshmello, Manuel Turizo",El Merengue,6sU751LOdNBPvVErW1GunP,51FvjPEGKq2zByeeEQ43V9,91,2023-03-03,189357,False,https://open.spotify.com/track/51FvjPEGKq2zBye...,...,0.677,8,-4.703,0,0.0442,0.03130,0.005170,0.1120,0.698,124.011


In [17]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [19]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [20]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [21]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [22]:
input_song_name = "I'm Good (Blue)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'I'm Good (Blue)':
                       Track Name                                     Artists  \
0                           REACT  Switch Disco, Ella Henderson, Robert Miles   
2                    Call It Love                     Felix Jaehn, Ray Dalton   
4  Where Did You Go? (feat. MNEK)                             Jax Jones, MNEK   
1                   Where You Are                          John Summit, Hayla   
3           Rainfall (Praise You)                                   Tom Santa   

                      Album Name Release Date  Popularity  
0                          REACT   2023-01-13        83.0  
2                   Call It Love   2022-09-16        83.0  
4  Where Did You Go (feat. MNEK)   2022-02-04        81.0  
1                  Where You Are   2023-03-03        79.0  
3          Rainfall (Praise You)   2022-02-18        77.0  


<ipython-input-21-3de633f34e34>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hybrid_recommendations = hybrid_recommendations.append({
